<a href="https://colab.research.google.com/github/keonju2/2022_DL_project/blob/main/yolov5_%EC%A0%9C%EC%9A%B02_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#model : yolo v5l
#batch size : 64
#image size : 640


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt  # install
!pip install -q roboflow

import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")


Cloning into 'yolov5'...
remote: Enumerating objects: 12128, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 12128 (delta 1), reused 0 (delta 0), pack-reused 12122
Receiving objects: 100% (12128/12128), 11.93 MiB | 5.64 MiB/s, done.
Resolving deltas: 100% (8395/8395), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 26.6 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 145 kB 40.1 MB/s 
     |████████████████████████████████| 178 kB 87.4 MB/s 
     |████████████████████████████████| 1.1 MB 66.7 MB/s 
     |████████████████████████████████| 67 kB 7.2 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 138 kB 79.1 MB

In [7]:
from zipfile import ZipFile
import os

def unzip(source_file, dest_path):
    with ZipFile(source_file, 'r') as zf:
        zipInfo = zf.infolist()
        for member in zipInfo:
            try:
                member.filename = member.filename.encode('cp437').decode('euc-kr', 'ignore')
                zf.extract(member, dest_path)
            except:
                raise Exception('what?!')

In [8]:
file_names = '/content/gdrive/MyDrive/DLP/Dataset/Training/[원천]도심로.zip'
unzip(file_names,'/content/yolov5/custom_dataset/train/images')

In [9]:
file_names = '/content/gdrive/MyDrive/DLP/Dataset/Valid/원천데이터.zip'
unzip(file_names,'/content/yolov5/sample_data')

In [10]:
shutil.move('/content/yolov5/sample_data/bb',
            '/content/yolov5/custom_dataset/val/images')

'/content/yolov5/custom_dataset/val/images'

In [11]:
shutil.rmtree('/content/yolov5/sample_data')

In [12]:
file_names = '/content/gdrive/MyDrive/DLP/Dataset/Valid/bb.zip'
unzip(file_names,'/content/yolov5/sample_data')

In [13]:
shutil.move('/content/yolov5/sample_data/bb','/content/yolov5/custom_dataset/val/labels')

'/content/yolov5/custom_dataset/val/labels'

In [14]:
file_names = '/content/gdrive/MyDrive/DLP/Dataset/Training/labels.zip'
unzip(file_names,'/content/yolov5/sample_data')

In [15]:
shutil.move('/content/yolov5/sample_data/labels',
            '/content/yolov5/custom_dataset/train/labels')

'/content/yolov5/custom_dataset/train/labels'

In [16]:
import yaml

make_yaml = {
             "train" : "/content/yolov5/custom_dataset/train",
            "val": "/content/yolov5/custom_dataset/val",

             "nc" : 32,
             "names": ['Vehicle_Car', 'Vehicle_Bus', 'Vehicle_Motorcycle', 'Vehicle_Unknown', 
                       'Pedestrian_Pedestrian', 'Pedestrian_Bicycle', 'Lane_White_Dash', 'Lane_White_Solid', 
                       'Lane_Yellow_Dash', 'Lane_Yellow_Solid', 'Lane_Blue_Dash', 'Lane_Blue_Solid', 
                       'TrafficLight_Red', 'TrafficLight_Yellow', 'TrafficLight_Green', 'TrafficLight_Arrow', 
                       'TrafficLight_RedArrow', 'TrafficLight_YellowArrow', 'TrafficLight_GreenArrow', 
                       'TrafficSign_Speed', 'TrafficSign_Else', 'RoadMark_StopLine', 'RoadMark_Crosswalk', 
                       'RoadMark_Number', 'RoadMark_Character', 'RoadMarkArrow_Straight', 'RoadMarkArrow_Left', 
                       'RoadMarkArrow_Right', 'RoadMarkArrow_StraightLeft', 'RoadMarkArrow_StraightRight', 
                       'RoadMarkArrow_Uturn', 'RoadMarkArrow_Else']
}
with open('/content/yolov5/data/custom_dataset.yaml', 'w') as outfile:
    yaml.dump(make_yaml, outfile, default_flow_style=False)

In [17]:
lab =['Vehicle_Car', 'Vehicle_Bus', 'Vehicle_Motorcycle', 'Vehicle_Unknown', 
                       'Pedestrian_Pedestrian', 'Pedestrian_Bicycle', 'Lane_White_Dash', 'Lane_White_Solid', 
                       'Lane_Yellow_Dash', 'Lane_Yellow_Solid', 'Lane_Blue_Dash', 'Lane_Blue_Solid', 
                       'TrafficLight_Red', 'TrafficLight_Yellow', 'TrafficLight_Green', 'TrafficLight_Arrow', 
                       'TrafficLight_RedArrow', 'TrafficLight_YellowArrow', 'TrafficLight_GreenArrow', 
                       'TrafficSign_Speed', 'TrafficSign_Else', 'RoadMark_StopLine', 'RoadMark_Crosswalk', 
                       'RoadMark_Number', 'RoadMark_Character', 'RoadMarkArrow_Straight', 'RoadMarkArrow_Left', 
                       'RoadMarkArrow_Right', 'RoadMarkArrow_StraightLeft', 'RoadMarkArrow_StraightRight', 
                       'RoadMarkArrow_Uturn', 'RoadMarkArrow_Else']

In [18]:
with open('/content/yolov5/w+labels.txt','w') as f:
  for i in lab:
    if i != lab[-1]:
      f.write(i+'\n')
    else:
      f.write(i)

In [19]:
for (root, directories, files) in os.walk('/content/yolov5/custom_dataset'):
    for file in files:
        file_path = os.path.join(root, file)
        change_file_path = file.replace('_v001_1','')
        file_path2 = os.path.join(root, change_file_path)
        os.rename(file_path,file_path2)

In [20]:
for (root, directories, files) in os.walk('/content/yolov5/custom_dataset'):
    for file in directories:
        file_path = os.path.join(root, file)
        if 'annotations' in file_path:
          change_file_path = file_path.replace('_annotations_v001_1','')
          
          shutil.move(file_path,change_file_path)
        

In [21]:
torch.cuda.empty_cache()

In [26]:
!python train.py --img 640 --cfg yolov5l.yaml  --batch 32 --epochs 1 --data /content/yolov5/data/custom_dataset.yaml --weights '' 

train: weights=, cfg=yolov5l.yaml, data=/content/yolov5/data/custom_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=1, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-236-gdcf8073 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, 

In [23]:
import shutil
save_path = '/content/gdrive/MyDrive/DLP/result/제우2'
save_file_path = '/content/yolov5/runs/train'

shutil.move(save_file_path, save_path)

Error: ignored

In [ ]:
!python detect.py --source '/content/gdrive/MyDrive/traffic_pic.jpg' --weight '/content/gdrive/MyDrive/DLP/result/제우2/train/exp/weights/best.pt'